In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import ast

# Load the data
input_df = pd.read_csv('candidates_with_questions.csv')


In [7]:
# Process the answers column to get a numeric matrix
def extract_answers(row):
    # Convert string representation of list to actual list
    answers = ast.literal_eval(row['answers'])
    # Create dictionary with questionId as key and value as value
    return {item['questionId']: item['value']/100 for item in answers}

# Apply extraction to each row
candidates_answers = input_df.apply(extract_answers, axis=1)

# Create a dataframe with all questions
question_ids = set()
for answers in candidates_answers:
    question_ids.update(answers.keys())
question_ids = sorted(question_ids)


In [8]:
# Create feature matrix
X = np.zeros((len(input_df), len(question_ids)))
for i, answers in enumerate(candidates_answers):
    for j, q_id in enumerate(question_ids):
        if q_id in answers:
            X[i, j] = answers[q_id]


In [9]:
print(X.shape[1])

58


In [10]:
input_dim = X.shape[1]
encoding_dim = 4  # Number of dimensions for smart spider

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.GELU(),
            nn.Linear(32, 16),
            nn.GELU(),
            nn.Linear(16, encoding_dim),
            nn.Sigmoid()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 16),
            nn.GELU(),
            nn.Linear(16, 32),
            nn.GELU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Prepare data
torch_X = torch.tensor(X, dtype=torch.float32).to(device)
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(torch_X, test_size=0.2, random_state=42)

# Model, loss, optimizer
model = AutoEncoder(input_dim, encoding_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Training loop
num_epochs = 100
batch_size = 64
for epoch in range(num_epochs):
    model.train()
    perm = torch.randperm(X_train.size(0))
    for i in range(0, X_train.size(0), batch_size):
        idx = perm[i:i+batch_size]
        batch = X_train[idx]
        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 1/100, Loss: 0.6468
Epoch 2/100, Loss: 0.5321
Epoch 3/100, Loss: 0.5128
Epoch 4/100, Loss: 0.4942
Epoch 5/100, Loss: 0.4565
Epoch 6/100, Loss: 0.4497
Epoch 7/100, Loss: 0.4995
Epoch 8/100, Loss: 0.4777
Epoch 9/100, Loss: 0.4738
Epoch 10/100, Loss: 0.4759
Epoch 11/100, Loss: 0.4389
Epoch 12/100, Loss: 0.4599
Epoch 13/100, Loss: 0.5257
Epoch 14/100, Loss: 0.5038
Epoch 15/100, Loss: 0.5168
Epoch 16/100, Loss: 0.4287
Epoch 17/100, Loss: 0.4561
Epoch 18/100, Loss: 0.4220
Epoch 19/100, Loss: 0.4397
Epoch 20/100, Loss: 0.4400
Epoch 21/100, Loss: 0.5073
Epoch 22/100, Loss: 0.4801
Epoch 23/100, Loss: 0.4592
Epoch 24/100, Loss: 0.4782
Epoch 25/100, Loss: 0.4182
Epoch 26/100, Loss: 0.4351
Epoch 27/100, Loss: 0.4424
Epoch 28/100, Loss: 0.4331
Epoch 29/100, Loss: 0.4451
Epoch 30/100, Loss: 0.4428
Epoch 31/100, Loss: 0.4047
Epoch 32/100, Loss: 0.4192
Epoch 33/100, Loss: 0.4289
Epoch 34/100, Loss: 0.4284
Epoch 35/100, Loss: 0.4135
Epoch 36/100, Loss: 0.4132
Epoch 37/100, Loss: 0.4327
Epoch 38/1

In [11]:
# Get encoded features for test set
model.eval()
with torch.no_grad():
    test_encoded_features = model.encoder(X_test).cpu().numpy()
    test_last_layer = model(X_test).cpu().numpy()
    all_encoded_features = model.encoder(torch_X).cpu().numpy()

In [12]:
mismatch = np.average(np.abs(X_test.cpu().numpy() - test_last_layer))
print(f"Average mismatch: {mismatch:.4f}")

Average mismatch: 0.1702


In [13]:
features_df = pd.DataFrame(all_encoded_features)
id_names = input_df[["id", "firstname","lastname"]]
joined_df = pd.concat([id_names, features_df], axis=1)

joined_df = joined_df.rename(columns={"firstname": "first_name", "lastname": "last_name"})

joined_df = joined_df.sort_values(by=["first_name", "last_name"]).reset_index(drop=True)

joined_df.to_csv("spiders_smartvote.csv", index=False)
torch.save(model.state_dict(), "neural_net_1.pt")